# Training & Evaluation
* We are going to break down training and evaluation into multiple notebooks, one for each algorithm that we train and evalutate. 
* In this first notebook, we'll create baseline models to get the predictions based on `stratified` and `most frequent` classes

## Install Libraries

In [1]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


## Import Libraries

In [2]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from pathlib import Path

# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath(os.path.join('..'))

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.utils import preprocessing

## Initialize Directories

In [3]:
data_root_dir = Path("..", "data/")
models_root_dir = Path("..", "models/")

## Read Data

In [6]:
X_train = pd.read_csv(Path(data_root_dir,"X_train.csv"))
y_train = pd.read_csv(Path(data_root_dir,"y_train.csv"))
X_test = pd.read_csv(Path(data_root_dir,"X_test.csv"))
y_test = pd.read_csv(Path(data_root_dir,"y_test.csv"))


## BaseLine Models

### Stratified Classifier

In [13]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score,recall_score,precision_score,precision_recall_curve


dummy_stratified = DummyClassifier(strategy="stratified", random_state=42)
dummy_stratified.fit(X_train,y_train)

## predict and evaulate
stratified_predictions = dummy_stratified.predict(X_test)
stratified_recall = recall_score(y_test, stratified_predictions)
stratified_precision = precision_score(y_test, stratified_predictions)
stratified_f1 = f1_score(y_test, stratified_predictions)
print(f"Stratified Prediction - Recall: {stratified_recall:.4f}")
print(f"Stratified Prediction - Precision: {stratified_precision:.4f}")
print(f"Weighted F1: {stratified_f1:.4f}")

Stratified Prediction - Recall: 0.5936
Stratified Prediction - Precision: 0.5918
Weighted F1: 0.5927


Observations:
* Our `0.59` recall score tells us that out of all positive cases our model was able to identify 59% of cases,i.e it misdiagnoized 41% cases where the students were actually depressed.
* `0.59` Precision score tells us that out of all the postive predictions only 59% were actually positive and 41% were false positive, i.e. 41% of the students will need unnecessary worry, follow-up and resource allocation. 

In [14]:

dummy_most_frequent = DummyClassifier(strategy="most_frequent", random_state=42)
dummy_most_frequent.fit(X_train,y_train)

## predict and evaulate
most_frequent_predictions = dummy_most_frequent.predict(X_test)
most_frequent_recall = recall_score(y_test, most_frequent_predictions)
most_frequent_precision = precision_score(y_test, most_frequent_predictions)
most_frequent_f1 = f1_score(y_test, most_frequent_predictions)
print(f"Most Frequent Prediction - Recall: {most_frequent_recall:.4f}")
print(f"Most Frequent Prediction - Precision: {most_frequent_precision:.4f}")
print(f"Weighted F1: {most_frequent_f1:.4f}")

Most Frequent Prediction - Recall: 1.0000
Most Frequent Prediction - Precision: 0.5856
Weighted F1: 0.7386


Observations:
* Our `1` recall score tells us that out of all positive cases our model was able to identify 100% of cases,which is great because our model didn't misdiagnoize any actually depressed students.
* `0.58` Precision score tells us that out of all the postive predictions only 58% were actually positive and 42% were false positive, i.e. 42% of the students will need unnecessary worry, follow-up and resource allocation. 
* Even though recall score of 1 is desirable, we want to aim for better F1 score, i.e better balance between recall and precision. 